# 1. Instalación e importación de librerias

### Instalación:

In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud tqdm
!pip install spacy-langdetect
!python -m spacy download es_core_news_sm


Defaulting to user installation because normal site-packages is not writeable
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ------------------------ --------------- 7.3/11.8 MB 37.6 MB/s eta 0:00:01
   ---------------------------------------- 11.8/11.8 MB 33.5 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   --------------------------------------- 632.6/632.6 kB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 25.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 34.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ---------------------------------------- 5.4/5.4 MB 41.0 MB/s eta

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.4.2 which is incompatible.
ydata-profiling 4.12.2 requires visions[type_image_path]<0.8.0,>=0.7.5, but you have visions 0.7.4 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/998.1 kB ? eta -:--:--
     ------------------------------------- 998.1/998.1 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993436 sha256=e48cc2ace7a7beb8aa13d9f6bc9f371a2ed4acf5fdc84667423f07237620922b
  Stored in directory: c:\users\mgs05\appdata\local\pip\cache\wheels\bc\5d\e7\dfa384408d6ca60501788f512a6d4f6e73cc60f3ad7c75faee
Successfully built langdetect
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ------------------------ --------------- 7.9/12.9 MB 44.2 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 38.5 MB/s eta 0:00:00
✔ Download and installation successful


In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Procesamiento de texto
import re
import string
import nltk
import spacy
from nltk.corpus import stopwords
from spacy_langdetect import LanguageDetector
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Progreso en bucles
from tqdm import tqdm

# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords de NLTK en español
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 2. Perfilamiento y entendimiento de los datos

### Carga de datos

In [19]:
# Cargar datos
data=pd.read_csv('fake_news_spanish.csv', sep=';', encoding = "utf-8")

news_df = data.copy()

In [20]:
news_df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


In [21]:
news_df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57063 entries, 0 to 57062
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           57063 non-null  object
 1   Label        57063 non-null  int64 
 2   Titulo       57047 non-null  object
 3   Descripcion  57063 non-null  object
 4   Fecha        57063 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


# 3. Preparación de los datos

### 3.1. Limpieza de datos.

In [25]:
def remove_non_ascii(words):
    """Remueve caracteres no ASCII de una lista de palabras"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convierte todas las palabras a minúsculas"""
    return [word.lower() for word in words if word is not None]

def remove_punctuation(words):
    """Elimina signos de puntuación"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if word is not None and re.sub(r'[^\w\s]', '', word) != '']

def remove_numbers(words):
    """Elimina los números de la lista de palabras"""
    return [word for word in words if not word.isdigit()]

def remove_stopwords(words):
    """Elimina stopwords en español"""
    return [word for word in words if word not in stopwords_es]

def preprocessing(text):
    """Aplica todas las funciones de limpieza de texto a un string"""
    words = text.split()  # Tokenización básica separando por espacios
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return " ".join(words)  # Retorna el texto limpio como un string


In [27]:
# Aplicar la limpieza a la columna 'Titulo' y 'Descripcion'
news_df['Titulo_Limpio'] = news_df['Titulo'].astype(str).apply(preprocessing)
news_df['Descripcion_Limpia'] = news_df['Descripcion'].astype(str).apply(preprocessing)

# Compración antes y después de la limpieza
print(news_df[['Titulo', 'Titulo_Limpio']].head())
print(news_df[['Descripcion', 'Descripcion_Limpia']].head())


                                              Titulo  \
0  'The Guardian' va con Sánchez: 'Europa necesit...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  Puigdemont: 'No sería ninguna tragedia una rep...   

                                       Titulo_Limpio  
0  the guardian va sanchez europa necesita apuest...  
1  revelan gobierno negocio liberacion mireles ca...  
2  ahora nunca joan fuster estatuto valenciano cu...  
3  iglesias alienta yolanda diaz erc eh bildu neg...  
4  puigdemont seria ninguna tragedia repeticion e...  
                                         Descripcion  \
0  El diario británico publicó este pasado jueves...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El valencianismo convoca en Castelló su fiesta...   
3  En política, igual que hay que negociar con lo...   
4  En una entrevista en El Punt Avui, el líder de... 

### 3.2. Tokenización

In [30]:
import nltk
nltk.download('punkt')  # Descarga el tokenizer de NLTK


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')  # Descarga el tokenizer de NLTK
def tokenize_nltk(text):
    """Tokeniza el texto usando NLTK"""
    return word_tokenize(text, language="spanish")  # Especificamos español

# Aplicar tokenización en el DataFrame
news_df['Titulo_Tokens'] = news_df['Titulo'].astype(str).apply(tokenize_nltk)
news_df['Descripcion_Tokens'] = news_df['Descripcion'].astype(str).apply(tokenize_nltk)

# Mostrar ejemplos
print(news_df[['Titulo', 'Titulo_Tokens']].head())


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


                                              Titulo  \
0  'The Guardian' va con Sánchez: 'Europa necesit...   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  Puigdemont: 'No sería ninguna tragedia una rep...   

                                       Titulo_Tokens  
0  ['The, Guardian, ', va, con, Sánchez, :, 'Euro...  
1  [REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...  
2  [El, 'Ahora, o, nunca, ', de, Joan, Fuster, so...  
3  [Iglesias, alienta, a, Yolanda, Díaz, ,, ERC, ...  
4  [Puigdemont, :, 'No, sería, ninguna, tragedia,...  


### 3.3. Normalización